# Section 1-0 - Prelude

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('titanic.csv')

In [3]:
df_train = df.iloc[:712, :]

df_train = df_train.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

age_mean = df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(age_mean)
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1}).astype(int)

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
scaler = StandardScaler()

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [4]:
df_test = df.iloc[712:, :]

df_test = df_test.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values

 ## Benchmark

In [5]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, y_train)

y_prediction = model.predict(X_test)
print "\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.832402234637


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## 1-layer Neural Network

In [10]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum()

In [16]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W = np.random.rand(2, 6) / 10
    b = np.random.rand(2,) / 10

    scores = []
    loss = 0
    
    for j in xrange(X_train.shape[0]):
        result = np.dot(W, X_train[j]) + b
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])

    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))
    
    if loss < min_loss:
        min_loss = loss
        best_weights = (W, b)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.685 accuracy 0.64 loop 0
loss 0.683 accuracy 0.642 loop 1
loss 0.681 accuracy 0.695 loop 5
loss 0.675 accuracy 0.712 loop 8
loss 0.673 accuracy 0.737 loop 44
loss 0.661 accuracy 0.739 loop 47
loss 0.658 accuracy 0.715 loop 297
loss 0.652 accuracy 0.77 loop 514

time taken 8.20281195641 seconds


In [17]:
W, b = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W, X_test[j]) + b
    result = softmax(result)
    scores.append(list(result))

y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.810055865922


## 2-layer Neural Network

In [21]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 6) / 10
    b_1 = np.random.rand(100,) / 10
    W_2 = np.random.rand(2, 100) / 10
    b_2 = np.random.rand(2,) / 10
    
    scores = []
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])

    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))
        
    if loss < min_loss:
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.682 accuracy 0.61 loop 0
loss 0.679 accuracy 0.757 loop 5
loss 0.678 accuracy 0.657 loop 8
loss 0.675 accuracy 0.787 loop 15
loss 0.67 accuracy 0.676 loop 29
loss 0.668 accuracy 0.633 loop 333
loss 0.663 accuracy 0.702 loop 362
loss 0.663 accuracy 0.708 loop 590

time taken 9.62385892868 seconds


In [22]:
W_1, b_1, W_2, b_2 = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = softmax(result)
    scores.append(list(result))
    
y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.670391061453


## 3-layer Neural Network

In [23]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 6) / 10
    b_1 = np.random.rand(100,) / 10
    W_2 = np.random.rand(100, 100) / 10
    b_2 = np.random.rand(100,) / 10
    W_3 = np.random.rand(2, 100) / 10
    b_3 = np.random.rand(2,) / 10
    
    scores = []
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = np.dot(W_3, result) + b_3
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])
        
    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))          
        
    if loss < min_loss:
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2, W_3, b_3)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.797 accuracy 0.287 loop 0
loss 0.753 accuracy 0.296 loop 1
loss 0.752 accuracy 0.285 loop 2
loss 0.695 accuracy 0.39 loop 4
loss 0.656 accuracy 0.705 loop 5
loss 0.644 accuracy 0.709 loop 7
loss 0.642 accuracy 0.705 loop 12
loss 0.633 accuracy 0.709 loop 19
loss 0.621 accuracy 0.721 loop 79
loss 0.621 accuracy 0.715 loop 130
loss 0.619 accuracy 0.728 loop 501
loss 0.618 accuracy 0.715 loop 956

time taken 13.7536799908 seconds


In [24]:
W_1, b_1, W_2, b_2, W_3, b_3 = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = np.dot(W_3, result) + b_3    
    result = softmax(result)
    scores.append(list(result))
    
y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.703910614525
